# Classify patches from D&F images.
Classify patches from tumor B13, analyzed in notebook 006. 

In [1]:
import time
import os
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_OUT = "/home/jrm/Martinez/images/temp/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG
IMAGE_FILENAME='B13.DF1.115.tif'

In [2]:
def get_image_names(path,pattern):
    #RAW_IMAGE_NAMES = os.listdir(DIR_IMAGES_RAW)
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)
#FILENAMES_IMAGES_RAW

In [3]:
def load_pixel_array(path,filename,verbose=False):
    im = Image.open(path+filename)
    ima = np.array(im)   # convert to numpy
    if verbose:
        print(filename, ima.size, ima.shape)
    return ima
pixel_array = load_pixel_array(DIR_IMAGES_RAW,IMAGE_FILENAME,True)

B13.DF1.115.tif 479598264 (12513, 12776, 3)


In [4]:
avg = np.average(pixel_array,axis=(0,1))
std = np.std(pixel_array,axis=(0,1))
print("Average",avg,"STD",std)
print("Example pixel:",pixel_array[0,0])

Average [ 0.         17.85214838 35.88764711] STD [ 0.         14.50691847 48.9509621 ]
Example pixel: [ 0 13  3]


In [5]:
#PATCH_THRESHOLD=PATCH_SIZE*PATCH_SIZE/2
IM_WIDTH = pixel_array.shape[0]
IM_HEIGHT = pixel_array.shape[1]

In [6]:
# TO DO: Create an iterator class and dispense with the globals.
W=0
H=0
def get_next_patch(pixels): # assume square
    global W,H
    if W+PATCH_SIZE>IM_WIDTH:
        H += PATCH_SIZE
        W = 0
    if H+PATCH_SIZE>IM_HEIGHT:
        patch = None
    if W+PATCH_SIZE<=IM_WIDTH and H+PATCH_SIZE<=IM_HEIGHT:
        patch = pixels[W:W+PATCH_SIZE, H:H+PATCH_SIZE]
        W += PATCH_SIZE
    return patch

In [7]:
def pixel_to_heatmap(green,blue):   # TO DO: this is slow, need a hash function
    bins=[10,20,30,40,50,60,70,80,90,256]
    gbin=None
    bbin=None
    for bin in range(0,10):
        if gbin is None and green<=bins[bin]:
            gbin=bin
        if bbin is None and blue<=bins[bin]:
            bbin=bin
        if gbin is not None and bbin is not None:
            return gbin,bbin
    return gbin,bbin
def accumulate_pixels(imary,verbose=False):  # TO DO: nested for loop is very slow
    heatmap=np.zeros( (10,10), dtype=np.int32)
    if verbose:
        print("accumulate",imary.shape)
    nrows,ncols,nchannel=imary.shape
    for row in range(0,nrows):
        for col in range(0,ncols):
            pixel = imary[row,col]
            #red = pixel[0]
            green = pixel[1]
            blue = pixel[2]
            gbin,bbin = pixel_to_heatmap(green,blue)
            heatmap[gbin,bbin] += 1
    return heatmap

In [13]:
def save_patch (patch,name):
    pic = Image.fromarray(patch)
    pic.save(DIR_IMAGES_OUT+name)  
def show_heatmap (patch,name):
    hm = accumulate_pixels(patch)
    print(hm,name)        

In [19]:
W=0
H=0
NO_GREEN_INTENSITY=np.array( [0,30,0] )
NO_BLUE_INTENSITY=np.array( [0,0,30] )
num_green_patches = 0
num_blue_patches = 0
num_patches = 0
print("Processing...")
patch = get_next_patch(pixel_array)
while patch is not None:
    num_patches += 1
    num_green_pixels = np.sum(np.all(patch >= NO_GREEN_INTENSITY,axis=2))
    num_blue_pixels = np.sum(np.all(patch >= NO_BLUE_INTENSITY,axis=2))
    #print(num_patches,num_green_pixels,num_blue_pixels)
    if num_blue_pixels>=10000 and num_green_pixels<1000:
        num_blue_patches += 1
        if num_blue_patches < 10:  # in [250,256,260]:
            name = "BLUE{}.tif".format(num_blue_patches)
            save_patch(patch,name)
            show_heatmap(patch,name)
    elif num_green_pixels>=10000 and num_blue_pixels<10000:
        num_green_patches += 1
        if num_green_patches < 10:
            name = "GREEN{}.tif".format(num_green_patches)
            save_patch(patch,name)
            show_heatmap(patch,name)

    patch = get_next_patch(pixel_array)
print("green=",num_green_patches,"blue=",num_blue_patches,"total=",num_patches)
print("green=",num_green_patches/num_patches,"blue=",num_blue_patches/num_patches)

Processing...
[[14523     0     0     0     0     0     0     0     0     0]
 [18409   330     6     0     1     0     0     0     0     0]
 [ 4027  1229   263    57    22    11     1     0     0     0]
 [ 2574  1276    92    35    28    63    51    23     1     0]
 [  863  4002    92    73    61    69    45    43    45     2]
 [   93  1222    65    50    34    21    22    21     6     0]
 [    8   180    48    32    12     7     5     1     0     0]
 [    1     5     7     7     1     0     0     0     0     0]
 [    2     0     0     0     0     0     0     0     0     0]
 [    9     0     0     0     0     0     0     0     0     0]] GREEN1.tif
[[8300  139    1    0    0    0    0    0    0    0]
 [7861  392    1    0    0    0    0    0    0    0]
 [9936 2425   13    0    0    0    0    0    0    0]
 [9066 6960   37    0    0    0    0    0    0    0]
 [1744 1962    2    0    0    0    0    0    0    0]
 [ 347  672    0    0    0    0    0    0    0    0]
 [  18  191    0    0    0

[[    0   235 16456  1820   113    17     1     0     0     0]
 [    0   169  6424  5532  2341  1507  1177   933   653  1167]
 [    0     0     0   241   496   468   458   621   768  8203]
 [    0     0     0     0     0     1     6     5     9   354]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     1]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]] BLUE5.tif
[[  285   801    62     0     0     0     0     0     0     0]
 [ 1222 18356  9662  2448  1209   880   732   692   614  1106]
 [    0  2823  1768   722   434   426   421   536   605  3831]
 [    0    27    18    67    40    32    34    42    46   229]
 [    0     0     0     1     0     0     0     0     1     4]
 [    0     0     0     0     0     0     0 